In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt

In [3]:
import os
path = os.getcwd()
samples = pd.read_csv(path + "./sample2000.csv")

In [18]:
import nltk

def stopword(documents):
    """불용어"""
    df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
    df[0] = df[0].apply(lambda x: x.strip())
    stopwords = df[0].to_numpy()
    nltk.download('punkt')
    for i, document in enumerate(documents):
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: 
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  

    return documents

import konlpy

def agan_wextraction(documents):
    """조사, 어미, 구두점 제외"""
    for i, document in enumerate(documents):
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): 
            if word[1] not in ['Josa', 'Eomi', 'Punctuation']: 
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        documents[i] = document
    return documents

def preprocessing(document):
    document = stopword(document)
    document = agan_wextraction(document)
    return document

In [ ]:
samples['document'] = preprocessing(samples['document'])

## counting 방식

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
train_data_features = vectorizer.fit_transform(samples['document'])

In [21]:
# 상위 5000개
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data_features, samples['label'], test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=42)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)

np.sum(y_test == y_pred_rf) / len(y_test)

# 53.5 -> 7225

0.7225

## IDF

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

IDF = TfidfVectorizer()
train_data_features = IDF.fit_transform(samples['document'])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data_features, samples['label'], test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=42)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)

np.sum(y_test == y_pred_rf) / len(y_test)

# 53.5 -> 7375

0.7375